In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\Sviridov\Pet project\DAU_WAU_MAU_Retention\events.csv", parse_dates=['ts'])
print(df.head())
print(df.info())

   user_id         ts     event
0        1 2025-06-23  purchase
1        1 2025-06-23      cart
2        1 2025-06-16      view
3        1 2025-06-16      view
4        1 2025-06-16      view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4690 entries, 0 to 4689
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   user_id  4690 non-null   int64         
 1   ts       4690 non-null   datetime64[ns]
 2   event    4690 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 110.0+ KB
None


In [3]:
print(f"{df['ts'].min()}")
print(f"{df['ts'].max()}")

2025-06-01 00:00:00
2025-06-30 00:00:00


In [4]:
# отрезаем данные за последний месяц 
cutoff = df['ts'].max() - pd.Timedelta(days=29)
df = df[df['ts'] >= cutoff]
df['ts'] = df['ts'].dt.floor('d')
df.sort_values(by='ts').head()

,user_id,ts,event
899,41,2025-06-01,purchase
2934,130,2025-06-01,purchase
2014,88,2025-06-01,view
2015,88,2025-06-01,view
1505,66,2025-06-01,view


In [5]:
# day d
d = df['ts'].max()

In [6]:
# dau_wau_mau
dwm = df.groupby('ts')['user_id'].nunique().reset_index()
dwm.columns = ['ts', 'dau']
dwm['wau'] = dwm['dau'].rolling(window=7,  min_periods=1).sum()
dwm['mau'] = dwm['dau'].rolling(window=30,  min_periods=1).sum()
print(dwm[dwm['ts'] == d])

           ts  dau    wau     mau
29 2025-06-30   57  352.0  1552.0


In [7]:
# retention
day_users = df[['user_id', 'ts']].drop_duplicates(['user_id', 'ts'])
d_users = day_users[day_users['ts'] == d]
d1_users = day_users[day_users['ts'] == d - pd.Timedelta(days=1)]
d7_users = day_users[day_users['ts'] == d - pd.Timedelta(days=7)]

d1_ret = d1_users['user_id'].isin(d_users['user_id']).sum()/d1_users['user_id'].count()
d7_ret = d7_users['user_id'].isin(d_users['user_id']).sum()/d7_users['user_id'].count()

In [8]:
# conversion
steps_table = df.sort_values(['user_id', 'ts'])
steps_table['next_event'] = steps_table.groupby('user_id')['event'].shift(-1)
steps_table.head()

,user_id,ts,event,next_event
2,1,2025-06-16,view,view
3,1,2025-06-16,view,view
4,1,2025-06-16,view,view
5,1,2025-06-16,view,view
6,1,2025-06-16,view,purchase


In [9]:
result = (
    steps_table
    .groupby(['event', 'next_event'])['user_id']
    .nunique()
    .reset_index()
)
result = result.loc[[1, 6, 7]].reset_index(drop=True)
result.columns = ['event', 'next_event', 'next_event_users']
result

,event,next_event,next_event_users
0,cart,purchase,68
1,view,cart,177
2,view,purchase,139


In [10]:
view_cart_users = df.groupby('event')['user_id'].nunique().reset_index()
view_cart_users

,event,user_id
0,cart,185
1,purchase,157
2,view,200


In [11]:
final = result.merge(view_cart_users, on='event', how='left')
final.columns = ['event', 'next_event', 'next_event_users', 'event_users']
final['conversion'] = final['next_event_users']/final['event_users']
final

,event,next_event,next_event_users,event_users,conversion
0,cart,purchase,68,185,0.367568
1,view,cart,177,200,0.885000
2,view,purchase,139,200,0.695000


In [12]:
print(f"DAU day_d: {dwm[dwm['ts'] == d]['dau'].iloc[0]:.0f}")
print(f"WAU day_d: {dwm[dwm['ts'] == d]['wau'].iloc[0]:.0f}")
print(f"MAU day_d: {dwm[dwm['ts'] == d]['mau'].iloc[0]:.0f}")

print(f"Retention d-1->d: {d1_ret:.2f}")
print(f"Retention d-7->d: {d7_ret:.2f}")

print(f"Conversion view->cart: {final['conversion'].iloc[0]:.2f}")
print(f"Conversion cart->purchase: {final['conversion'].iloc[1]:.2f}")
print(f"Conversion view->purchase: {final['conversion'].iloc[2]:.2f}")

DAU day_d: 57
WAU day_d: 352
MAU day_d: 1552
Retention d-1->d: 0.38
Retention d-7->d: 0.38
Conversion view->cart: 0.37
Conversion cart->purchase: 0.89
Conversion view->purchase: 0.69
